In [233]:
def getGamerInput(is_level, is_one_two):                          #функция для ввода
    gamer_input = input()
    if gamer_input == 'save' and (not is_level):
        return gamer_input
    if len(gamer_input) == 1:
        if gamer_input.isdigit() and 1 <= int(gamer_input) <= 3:
            if is_level:
                return int(gamer_input)
            elif is_one_two and int(gamer_input) <= 2:
                return int(gamer_input)
        elif (not is_level) and (not is_one_two) and gamer_input.isalpha():
            return gamer_input.lower()
    print('Error, try again')
    return getGamerInput(is_level, is_one_two)

In [234]:
def save(*elements):                                 #функция для сохранения
    with open('Save.txt', 'w') as save_file:
        save_file.writelines(elements) 

In [237]:
def serialization(element):
    with open("file.pkl", "wb") as fp:
        pickle.dump(element, fp)

In [238]:
def unserialization(element):
    with open(element, "rb") as fp:
        return pickle.load(fp)

In [240]:
class Letter():                                      #каждая буква как класс
    def __init__(self, letter, open_flag):
        self.letter = letter
        self.open_flag = open_flag
    
    def __getstate__(self) -> dict:  # Как мы будем "сохранять" класс
        state = {}
        state["letter"] = self.letter
        state["open_flag"] = self.open_flag
        return state

    def __setstate__(self, state: dict):  # Как мы будем восстанавливать класс из байтов
        self.letter = state["letter"]
        self.open_flag = state["open_flag"] 

    def show(self):
        if self.open_flag:
            return self.letter
        else:
            return '*'

In [241]:
class Slovo():                                                          # загаданное слово как класс
    def __init__(self, word, attemts_number, is_letter_in_word=False):
        self.word = word
        self.attemts_number = attemts_number
        self.is_letter_in_word = is_letter_in_word

        letter_list = [Letter(i, False) for i in word]
        self.letter_list = letter_list

        inputs_list = []
        self.inputs_list = inputs_list

    def __getstate__(self) -> dict:  # Как мы будем "сохранять" класс
        state = {}
        state["word"] = self.word
        state["attemts_number"] = self.attemts_number
        self.letter_list_dumps = pickle.dumps(self.letter_list)
        state["letter_list_dumps"] = self.letter_list_dumps
        state["inputs_list"] = self.inputs_list
        return state

    def __setstate__(self, state: dict):  # Как мы будем восстанавливать класс из байтов
        self.word = state["word"]
        self.attemts_number = state["attemts_number"]
        self.letter_list_dumps = state["letter_list_dumps"]
        self.letter_list = pickle.loads(self.letter_list_dumps)
        self.inputs_list = state["inputs_list"]

    def check(self, input_letter):
        if input_letter not in self.inputs_list:
            self.inputs_list.append(input_letter)
        else:
            print(f'You have already entered the letter "{input_letter}"')
            self.is_letter_in_word = True
        
    def decrease_attemts(self, input_letter):
        if not self.is_letter_in_word:
            print(f'The letter "{input_letter}" is not in the word')
            self.attemts_number -=1

    def show(self, show_word_flag, input_letter=None):
        if show_word_flag:
            new_word = self.word
            return new_word
        else:
            new_word = ''
            self.is_letter_in_word = False
            for i in self.letter_list:
                if i.letter == input_letter:
                    self.is_letter_in_word = True
                    i.open_flag = True
                new_word += i.show()
            return new_word

    def game_mechanics(self, show_word_flag, input_letter):
        new_word = self.show(show_word_flag, input_letter)
        self.check(input_letter)
        self.decrease_attemts(input_letter)
                
        return (new_word, self.attemts_number)
            


In [242]:
def edit_level_sentence(dict, level):                                                        #функция для длинных предложений
    return f'\n{dict[str(level)][0]} - The word consists of {dict[str(level)][1]} letters, \
the number of attempts is {dict[str(level)][2]}'

In [243]:
def game(levels_dict):                                         #функция игры
    print('New game - 1\nContinue game - 2')
    gamer_input = getGamerInput(False, True)
    if gamer_input == 2:
        slovo = unserialization("file.pkl")
        print(slovo.show(False))
                
    elif gamer_input == 1:
        print(f'\nSelect difficulty level:\
            {edit_level_sentence(levels_dict, 1)} (Press 1)\
            {edit_level_sentence(levels_dict, 2)} (Press 2)\
            {edit_level_sentence(levels_dict, 3)} (Press 3)')       #вывод уровней

        gamer_input = getGamerInput(True, False)                     # ввод уровня
        
    level = gamer_input
    print(edit_level_sentence(levels_dict, level))

    word = levels_dict[str(gamer_input)][3]
    attemts_number = levels_dict[str(gamer_input)][2]
    slovo = Slovo(word, attemts_number)
    input_number = 0

    print(slovo.show(False))
    while attemts_number != 0:
        gamer_input = getGamerInput(False, False)
        if gamer_input == 'save':
            print('\nSee you')
            return serialization(slovo)
        input_number += 1

        new_word, attemts_number = slovo.game_mechanics(False, gamer_input)
        print(f'{new_word} \nAttempts left: {attemts_number}')

        if new_word == slovo.show(True):
            print(f'\nCongratulations\nYou win\nYou completed the game in {input_number} turns')
            break
        
    if attemts_number == 0:         #если игрок проиграл
        print('\nGame over')

    print('\nDo you want to start a new game?\nYes - 1\nNo - 2')

    gamer_input = getGamerInput(False, True)
    if gamer_input == 1:
        game(levels_dict)
    else:
        print('Goodbye')
        

In [244]:
levels_dict = {'1': ['Easy', 5, 9, 'apple'], '2': ['Normal', 7, 7, 'shelter'], '3': ['Hard', 9, 7, 'binominal']}
game(levels_dict)

New game - 1
Continue game - 2
bi***i***

Normal - The word consists of 7 letters, the number of attempts is 7
*******
******r 
Attempts left: 7


KeyboardInterrupt: Interrupted by user

In [ ]:
import pickle